In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
cd /raid/home/myang349/recsys-filterbubbles/

/raid/home/myang349/recsys-filterbubbles


# Imports

In [188]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.optim as optim
from os import listdir
from os.path import isfile, join
from tracin.tracin import (
    save_tracin_checkpoint,
    load_tracin_checkpoint,
    calculate_tracin_influence,
)
import pandas as pd
from LSTM_clean.utils import filter_and_split_data, sequence_generator, printl
from LSTM_clean.model import LSTM 
from collections import Counter
import numpy as np
import pickle
from collections import defaultdict
import copy
from torch.optim import SGD
from lstm_wrapper import train_model, get_topk_predictions

# Loading Data

In [126]:
# Data Location
SAVE_FOLDER = "/raid/home/myang349/recsys-filterbubbles/data/twitch_sequence/"
SAVE_TRAIN_NAME = "train.data"
SAVE_VALID_NAME = "valid.data"
SAVE_TEST_NAME = "test.data"

In [196]:
train_data = np.load(os.path.join(SAVE_FOLDER, SAVE_TRAIN_NAME), allow_pickle=True)
valid_data = np.load(os.path.join(SAVE_FOLDER, SAVE_VALID_NAME), allow_pickle=True)

# Training

In [45]:
model = myang_train_lstm.main()

Device is cuda
Train: 71465, Test: 10231
Model is  LSTM(
  (item_emb): Embedding(5400, 128, padding_idx=0)
  (LSTM): LSTM(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=5400, bias=True)
)

Training and testing
train # = 71465, test # = 10231

Epoch 0	Train Loss: 0.01683359364302941	Test MRR: 0.0020169323878277317	Test Recall@10: 0.002150327436223243	Elapsed time: 1.993915319442749
saving checkpoint to /raid/home/myang349/recsys-filterbubbles/checkpoints/lstm_checkpoint_epoch0.pt
Epoch 5	Train Loss: 0.016641196638777334	Test MRR: 0.040412084205684845	Test Recall@10: 0.07408855439350992	Elapsed time: 7.634873151779175
Epoch 10	Train Loss: 0.016163602419432654	Test MRR: 0.045309320656974655	Test Recall@10: 0.08992278369660835	Elapsed time: 7.055970668792725
saving checkpoint to /raid/home/myang349/recsys-filterbubbles/checkpoints/lstm_checkpoint_epoch10.pt
Epoch 15	Train Loss: 0.015449316236559625	Test MRR: 0.04690807107393686	Test Recall@10: 0.09148665819567979	El

## Verifying Predictions

In [ ]:
# working
# 839, 860

In [199]:
i=839
k=15

# print(np.array(train_data[i][0]))
# print(train_data[i][1])

# Create embeddings
data = [
    valid_data[i][0],
    valid_data[i+1][0],
]

gt = [
    valid_data[i][1],
    valid_data[i+1][1]
]

for i, pt in enumerate(data):
    print(np.array(pt), gt[i])

embedded_data = []
for pt in data:
    embedded_data.append(model.item_emb(torch.LongTensor(pt).to(model.device)))
# Move to torch
embedded_data = torch.stack(embedded_data, dim=0).detach()

output, hidden = model.forward(embedded_data)
preds = torch.topk(output, k).indices.tolist()

[   0    0    0    0    0    0    0    0    0    0    0    0    0   16
 1563  867 1563   16 1563 1971 1353 1003 1853  867 1353 1972 1563 1973
  429   16 1563 1563 1854 1974   16 1563  429 1003  429 1563   16 1854
 1854 1938 1975 1353  428  428  867] 1976
[   0    0    0    0    0    0    0    0    0    0    0    0   16 1563
  867 1563   16 1563 1971 1353 1003 1853  867 1353 1972 1563 1973  429
   16 1563 1563 1854 1974   16 1563  429 1003  429 1563   16 1854 1854
 1938 1975 1353  428  428  867 1976] 545


In [200]:
print(np.array(preds))

[[ 16 235 165 113 296 297 288 180 294 156 234  22 515 301 333]
 [ 16 113 165 235 296 180 294 297 515  22 333 465 502 288 234]]


In [147]:
train_data[i][1]

36

In [148]:
data = [
    train_data[i][0]
]

In [149]:
# Create embeddings
embedded_data = []
for pt in data:
    embedded_data.append(model.item_emb(torch.LongTensor(pt).to(model.device)))
# Move to torch
embedded_data = torch.stack(embedded_data, dim=0).detach()

In [150]:
output, hidden = model.forward(embedded_data)

In [151]:
k=10

In [152]:
preds = torch.topk(output, k).indices.tolist()

In [153]:
preds

[[1, 3, 6, 40, 602, 679, 44, 41, 80, 604]]

In [112]:
preds

[[963, 1007, 224, 232, 40, 602, 44, 41, 462, 235],
 [1, 6, 9, 80, 427, 3, 4, 10, 558, 43],
 [235, 462, 234, 465, 662, 431, 1007, 169, 288, 115]]

In [ ]:
for i in range(len(data)):
    data[i][0] = model.item_emb(torch.LongTensor(data[i][0]).to(model.device))
#     train_labels.append(train[i][1])
# torch.stack(data, dim=0).detach()

In [26]:
output, hidden = model.forward(curr)
output1 = output.view(-1, model.num_items)
prob = nn.functional.softmax(output1, dim=1).data.cpu()
k = 50

## Item Extraction Method 1: Torch

In [ ]:
topk = torch.topk(output, k).indices.tolist()
print(topk[0])

## Item Extraction Method 2: Numpy

In [49]:
print(list(np.argsort(-prob.numpy())[0][:k]))

[44, 113, 212, 41, 1954, 667, 3220, 537, 69, 147, 16, 277, 502, 39, 341, 353, 66, 647, 140, 43, 888, 629, 460, 169, 1741, 70, 410, 54, 553, 64, 71, 215, 431, 2234, 428, 819, 46, 536, 928, 463, 380, 573, 925, 427, 1958, 1371, 72, 307, 2359, 711]


In [68]:
np_prob

array([[4.6355075e-05, 1.1106163e-02, 5.6383542e-05, ..., 2.5252750e-05,
        2.7392212e-05, 3.5436089e-05]], dtype=float32)